# Tutorial: DSPy and LanceDB Integration

This tutorial demonstrates the integration of DSPy with LanceDB to create a scalable and efficient data processing and querying system. Each section will guide you through the key steps involved, with explanations provided for the corresponding blocks of code.

### Introduction
In this notebook, we integrate DSPy, a powerful data science library, with LanceDB, a high-performance database designed for machine learning applications. This combination is particularly effective for managing, processing, and querying large datasets in machine learning workflows.

In [19]:
!pip install dspy lancedb torch tantivy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.8 MB/s eta 0:00:00


In [7]:
import re
import dspy
import torch
import lancedb

from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
from lancedb.rerankers import (
    LinearCombinationReranker,
)  # LanceDB hybrid search uses LinearCombinationReranker by default

Sets up the device (GPU if available, otherwise CPU) and initializes the "BAAI/bge-small-en-v1.5" embedding model using Hugging Face.

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed_model = (
    get_registry()
    .get("huggingface")
    .create(name="BAAI/bge-small-en-v1.5", device=device)
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

# LanceDB Configuration

In this process, we'll create a vector store by defining a schema that includes text data and their corresponding embedding vectors. We'll set up a class, Vectorstore, which initializes with context information and a database path, establishes a connection to LanceDB, and persists the context data into a database table if it doesn't already exist. Additionally, we'll implement a method to search this table using hybrid queries, retrieving and ranking the most relevant context blocks based on the input query. This setup enables efficient storage, retrieval, and querying of contextual data.

In [10]:
class Schema(LanceModel):
    text: str = embed_model.SourceField()
    vector: Vector(embed_model.ndims()) = embed_model.VectorField()


class Vectorstore:
    def __init__(
        self, context_information=None, db_path=None, tablename="context", chunk_size=50
    ):
        if context_information is None or db_path is None:
            raise ValueError("Both context_information and db_path must be provided")

        self.context_information = context_information
        self.db_path = db_path
        self.tablename = tablename
        self.chunk_size = chunk_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.embed_model = (
            get_registry()
            .get("huggingface")
            .create(name="BAAI/bge-small-en-v1.5", device=self.device)
        )
        self.db = lancedb.connect(self.db_path)
        self._persist_on_db()

    def split_text_into_chunks(self):
        """Splits the context information into chunks of the specified size."""
        words = self.context_information.split()
        return [
            " ".join(words[i : i + self.chunk_size])
            for i in range(0, len(words), self.chunk_size)
        ]

    def _persist_on_db(self):
        if self.tablename not in self.db.table_names():
            tbl = self.db.create_table(self.tablename, schema=Schema, mode="overwrite")
            contexts = [
                {"text": re.sub(r"\s+", " ", text)}
                for text in self.split_text_into_chunks()
            ]
            tbl.add(contexts)
        else:
            tbl = self.db.open_table(self.tablename)

        tbl.create_fts_index("text", replace=True)

    def search_table(self, query_string, query_type="hybrid", top_k=3):
        print(
            f"Searching table with query type: {query_type}, table: {self.tablename}, query: {query_string}"
        )
        reranker = LinearCombinationReranker(weight=0.7)
        tbl = self.db.open_table(self.tablename)
        rs = (
            tbl.search(query_string, query_type=query_type)
            .rerank(reranker=reranker)
            .limit(top_k)
            .to_list()
        )
        return "- context block: ".join([item["text"] for item in rs])

# DSPy Configuration

We'll begin by configuring DSPy with a language model to handle our natural language processing tasks. Here, we use an OpenAI model, specifically the `gpt-4o-mini`, to power our language model (LLM) within DSPy. This setup is flexible—while we're using an OpenAI model in this instance, it's also possible to run any local LLM that is compatible with DSPy. By using a tool like Ollama, you can easily switch to a local LLM by modifying the model configuration. This approach allows for adaptability depending on your computational resources or specific model preferences.

All models and comprehensive documentation for DSPy, including how to configure and use different language models, can be found [here](https://dspy-docs.vercel.app/docs/building-blocks/language_models).


In [11]:
llm = dspy.OpenAI(
    model="gpt-4o-mini",
)
dspy.configure(lm=llm)

In DSPy, a signature defines the structure and expected inputs and outputs for a task, serving as a blueprint that ensures consistency in data handling and task execution. By clearly specifying the inputs, outputs, and their types, signatures help maintain a standardized approach to implementing various tasks within your pipeline. This ensures that different components can interact seamlessly and that the data flows correctly through each step of the process.

For more detailed information about signatures and how to use them, you can explore the [DSPy documentation on signatures](https://dspy-docs.vercel.app/docs/building-blocks/signatures).

In [12]:
class GenerateAnswer(dspy.Signature):
    """As a chat assitant, generates an answer based on a query and given context chunks."""

    query = dspy.InputField(
        desc="The question or query to be answered, if context is not provided answers respectfully that cannot help with that question",
        type=str,
    )
    context_chunks = dspy.InputField(
        desc="List of relevant context chunks to answer the query", type=list
    )
    answer = dspy.OutputField(desc="The answer to the query, 5-20 words")
    answer_rationale = dspy.OutputField(
        desc="LM's reasoning before it generates the output"
    )

We're implementing a Retrieval-Augmented Generation (RAG) module, which is a method that enhances the generation of answers by retrieving relevant information from a knowledge base.

RAG works by first searching for relevant context using a vector-based search in LanceDB, a high-performance database optimized for storing and querying multi-modal data. The vectorstore in LanceDB enables efficient retrieval of context by matching the user's query with relevant chunks of information stored as vectors. Once the relevant context is retrieved, it is used to generate a more accurate and informed answer.

The RAG class integrates these steps: it retrieves context using the LanceDB Vectorstore and then generates the final answer using the ChainOfThought mechanism with the GenerateAnswer signature. This approach ensures that the model provides answers that are both contextually relevant and coherent, leveraging the power of vector-based search for precise and efficient information retrieval.

In [13]:
class RAG(dspy.Module):
    def __init__(self, context_information, db_path):
        super().__init__()
        if context_information is None or db_path is None:
            raise ValueError("Both context_information and db_path must be provided")

        self.vectorstore = Vectorstore(
            context_information=context_information, db_path=db_path
        )
        self.generate_answer = dspy.ChainOfThought(
            GenerateAnswer
        )  # Using signature defined above

    def forward(self, query):
        relevant_contexts = self.vectorstore.search_table(
            query_string=query, query_type="hybrid", top_k=3
        )
        prediction = self.generate_answer(query=query, context_chunks=relevant_contexts)

        return dspy.Prediction(
            query=query,
            context_chunks=relevant_contexts,
            answer=prediction.answer,
            answer_rationale=prediction.answer_rationale,
        )

# EvaluatorRAG Module

The EvaluateAnswer class defines a signature for evaluating the accuracy of an answer. It specifies the inputs and outputs necessary to assess the quality of the generated response. The evaluation considers the original query, the context chunks used to form the answer, the answer itself, and the rationale behind it. The output includes an accuracy metric (rated from 0 to 10) and a rationale metric, which provides insight into the reasoning process.

In [14]:
class EvaluateAnswer(dspy.Signature):
    """Returns a 0-10 metric that measures the accuracy of the provided answer based on the given context chunks and the rationale provided by the algorithm."""

    query = dspy.InputField(desc="The question or query to be answered.", type=str)
    context_chunks = dspy.InputField(
        desc="List of relevant context chunks used to answer the query.", type=list
    )
    answer = dspy.InputField(desc="The provided answer to the query.", type=str)
    answer_rationale = dspy.InputField(
        desc="The reasoning given by the language model for the answer.", type=str
    )
    accuracy_metric = dspy.OutputField(
        desc="0-10 number that represents a metric evaluating the accuracy of the answer based on the answer vs contexts provided",
        type=int,
    )
    rationale_metric = dspy.OutputField(desc="LM's metric reasoning", type=str)

The EvaluatorRAG class is a module designed to implement the evaluation process defined by the EvaluateAnswer signature. It initializes the evaluation mechanism and provides a method (forward) that takes in the query, context chunks, the generated answer, and its rationale. This method evaluates the accuracy of the answer and normalizes the resulting accuracy metric to ensure it's a usable number. The module then returns a prediction object that includes the original inputs along with the evaluated accuracy and rationale metrics, providing a comprehensive assessment of the answer's quality.

In [15]:
class EvaluatorRAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.evaluate_answer = dspy.ChainOfThought(
            EvaluateAnswer
        )  # Using the EvaluateAnswer signature

    def forward(self, query, context_chunks, answer, answer_rationale):
        evaluation = self.evaluate_answer(
            query=query,
            context_chunks=context_chunks,
            answer=answer,
            answer_rationale=answer_rationale,
        )

        # Normalize the accuracy metric to ensure it's always a number
        accuracy_metric = self.normalize_metric(evaluation.accuracy_metric)

        return dspy.Prediction(
            query=query,
            context_chunks=context_chunks,
            answer=answer,
            answer_rationale=answer_rationale,
            accuracy_metric=accuracy_metric,
            rationale_metric=evaluation.rationale_metric,
        )

    def normalize_metric(self, metric):
        if isinstance(metric, str):
            match = re.search(r"\d+", metric)
            if match:
                return int(match.group())
        return metric

# RAG_Assitant Module
The RAG_Assitant class encapsulates both the generation and evaluation of answers within a single chain of operations. It initializes the RAG module for retrieving and generating the answer, and the EvaluatorRAG module for assessing the quality of that answer.

In the process_question method, the class first processes the query using the RAG module to generate an answer and relevant context. This result is then passed to the EvaluatorRAG module, which evaluates the accuracy and reasoning behind the generated answer. The final output is a comprehensive dictionary that includes the query, the context chunks used, the generated answer, the reasoning behind the answer, and the evaluation metrics. This structured approach ensures that both the generation and evaluation steps are seamlessly integrated, providing a robust solution for answering and assessing queries.

In [16]:
class RAG_Assitant(dspy.Module):
    def __init__(self, context_information, db_path):
        super().__init__()
        self.rag = RAG(context_information=context_information, db_path=db_path)
        self.evaluator_rag = EvaluatorRAG()

    def process_question(self, query):
        # Get the initial result from RAG
        result = self.rag.forward(query)

        # Evaluate the result using EvaluatorRAG
        evaluation = self.evaluator_rag.forward(
            query=query,
            context_chunks=result.context_chunks,
            answer=result.answer,
            answer_rationale=result.answer_rationale,
        )

        # Return the evaluation results as a dictionary
        return {
            "query": evaluation.query,
            "context_chunks": evaluation.context_chunks,
            "answer": evaluation.answer,
            "answer_rationale": evaluation.answer_rationale,
            "accuracy_metric": evaluation.accuracy_metric,
            "rationale_metric": evaluation.rationale_metric,
        }

# Testing

### Define context for initialization

In [17]:
CONTEXT_DATA = """
Welcome to BeatyPets
Welcome to BeatyPets!
Your go-to store for all your pet care needs.
Located in Petville, PA, BeatyPets offers a variety of services for your furry friends.
From grooming and health check-ups to training and boarding, we are here to cater to your pets' needs.
BeatyPets is your trusted pet shop, providing comprehensive services for dogs, cats, birds, reptiles, and small mammals. We specialize in grooming, health check-ups, vaccinations, training, and boarding services. Our expert staff includes veterinarians, groomers, and trainers dedicated to the well-being of your pets.
Visit us at 123 Pet Lane, Petville, PA 12345. Contact us at 555-123-4567 or email us at contact@beatypets.com. Explore more at http://beatypets.com.
Authors: BeatyPets Team
Categories: Pets, Pet Care
Published: 20240709
Updated: 20240709
Source: http://beatypets.com

BeatyPets Business Hours
Business Hours:
Monday - Friday: 9 AM - 6 PM
Saturday: 10 AM - 4 PM
Sunday: Closed
BeatyPets operates from Monday to Saturday with varied hours. We are closed on Sundays. Our convenient hours ensure we are available when you need us for your pet care needs.
Authors: BeatyPets Team
Categories: Pets, Pet Care, Business Hours
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/hours

Popular Dog Breeds Served at BeatyPets
Popular Dog Breeds Served:
Labrador Retriever
French Bulldog
German Shepherd
Golden Retriever
Bulldog
At BeatyPets, we serve a variety of popular dog breeds, including Labrador Retrievers, French Bulldogs, German Shepherds, Golden Retrievers, and Bulldogs. Our services cater to the needs of these beloved breeds with specialized care.
Authors: BeatyPets Team
Categories: Pets, Dog Breeds
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/popular-dogs

BeatyPets Cat Policy
Cat Policy:
You can bring your cat to BeatyPets! We provide grooming and health check-up services specifically for cats.
At BeatyPets, we welcome cats and offer specialized services including grooming and health check-ups. Your cat is in good hands with our expert staff, who ensure their comfort and well-being.
Authors: BeatyPets Team
Categories: Pets, Cat Care
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/cat-policy

Meet Our Staff at BeatyPets
Meet Our Staff:
Dr. Sarah Johnson - Veterinarian, Small Animals
Michael Brown - Groomer, Dogs and Cats
Emily Davis - Trainer, Behavioral Training
Meet our dedicated staff at BeatyPets who are committed to providing exceptional care for your pets. Our team includes experienced professionals in veterinary care, grooming, and training.
Authors: BeatyPets Team
Categories: Pets, Staff
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/staff

Upcoming Events at BeatyPets
Upcoming Events:
Pet Adoption Day - August 15, 2024
Pet Health Seminar - September 10, 2024
Join us at BeatyPets for exciting upcoming events! From pet adoption days to health seminars, there's something for every pet owner. Mark your calendar and don't miss out!
Authors: BeatyPets Team
Categories: Pets, Events
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/events

Appointments Available at BeatyPets
Appointments Available:
Monday - Friday: 9 AM - 6 PM
Saturday: 10 AM - 4 PM
Sunday: Closed
Services requiring an appointment: grooming, veterinary check-ups, training.
BeatyPets offers appointments from Monday to Saturday for services such as grooming, veterinary check-ups, and training. We are closed on Sundays. Be sure to book your appointment in advance to secure availability.
Authors: BeatyPets Team
Categories: Pets, Pet Care, Appointments
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/appointments

Products for Sale at BeatyPets
Products for Sale:
- Pet toys
- Beds and crates
- Collars and leashes
- Hygiene products
- Food and treats
At BeatyPets, we offer a wide range of pet products, including toys, beds, crates, collars, leashes, hygiene products, and a variety of food and treats. Find everything your pet needs in one place.
Authors: BeatyPets Team
Categories: Pets, Products
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/products

Types of Pet Food at BeatyPets
Types of Pet Food:
- Dry food
- Wet food
- Dietary food
- Grain-free food
- Treats and snacks
BeatyPets offers a variety of pet foods, including dry, wet, dietary, and grain-free options. We also have a selection of treats and snacks to keep your pets happy.
Authors: BeatyPets Team
Categories: Pets, Food
Published: 20240709
Updated: 20240709
Source: http://beatypets.com/food

"""

## Let's create our assitant
This line of code initializes the RAG_Assitant module by passing in the necessary context information and database path. The context_information parameter, provided as CONTEXT_DATA, contains the data that will be used to generate and evaluate answers. The db_path parameter specifies the path to the LanceDB database where the context data is stored and managed. This initialization prepares the RAG_Assitant for processing queries by setting up the entire chain from context retrieval to answer evaluation.

In [20]:
assistant = RAG_Assitant(context_information=CONTEXT_DATA, db_path="./db_lancedb")

In [25]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [26]:
test_question = "Is it open on Tuesday?"
result = assistant.process_question(test_question)
print("Answer:", result["answer"])
print("Answer Rationale:", result["answer_rationale"])
print("Accuracy:", result["accuracy_metric"])
print("Accuracy Rationale:", result["rationale_metric"])

Searching table with query type: hybrid, table: context, query: Is it open on Tuesday?
Answer: Yes, BeatyPets is open on Tuesday.
Answer Rationale: The context confirms that BeatyPets operates Monday to Friday, making it open on Tuesday.
Accuracy: 10
Accuracy Rationale: The answer is fully supported by the context, confirming that BeatyPets is indeed open on Tuesday.


In [27]:
test_question = "Who is the veterinarian at BeatyPets?"
result = assistant.process_question(test_question)
print("Answer:", result["answer"])
print("Answer Rationale:", result["answer_rationale"])
print("Accuracy:", result["accuracy_metric"])
print("Accuracy Rationale:", result["rationale_metric"])

Searching table with query type: hybrid, table: context, query: Who is the veterinarian at BeatyPets?
Answer: Dr. Sarah Johnson is the veterinarian at BeatyPets.
Answer Rationale: The context clearly identifies Dr. Sarah Johnson as the veterinarian, allowing for a straightforward answer.
Accuracy: 10
Accuracy Rationale: The answer is fully supported by the context, and the rationale correctly explains the basis for the answer.


### Challenging questions
We are undertaking a series of challenging questions to rigorously test different metrics within our RAG and evaluation modules. This process will help ensure that the models are not only generating accurate responses but are also providing well-reasoned justifications and maintaining high standards of evaluation accuracy.

In [28]:
test_question = "Can BeatyPets handle aggressive dogs?"
result = assistant.process_question(test_question)
print("Answer:", result["answer"])
print("Answer Rationale:", result["answer_rationale"])
print("Accuracy:", result["accuracy_metric"])
print("Accuracy Rationale:", result["rationale_metric"])

Searching table with query type: hybrid, table: context, query: Can BeatyPets handle aggressive dogs?
Answer: The context does not specify handling aggressive dogs.
Answer Rationale: The reasoning process involved analyzing the context provided, which focused on the breeds served and staff expertise but lacked specific information about handling aggressive dogs.
Accuracy: 9
Accuracy Rationale: The answer accurately identifies the lack of information regarding aggressive dogs in the context, demonstrating a clear understanding of the provided material.


In [29]:
test_question = "Can I schedule a grooming appointment online?"
result = assistant.process_question(test_question)
print("Answer:", result["answer"])
print("Answer Rationale:", result["answer_rationale"])
print("Accuracy:", result["accuracy_metric"])
print("Accuracy Rationale:", result["rationale_metric"])

Searching table with query type: hybrid, table: context, query: Can I schedule a grooming appointment online?
Answer: It's unclear if you can schedule online.
Answer Rationale: The context mentions appointments are needed but does not specify if they can be scheduled online.
Accuracy: 7
Accuracy Rationale: The answer captures the uncertainty present in the context but could be improved by acknowledging


In [30]:
test_question = "Is BeatyPets open on public holidays?"
result = assistant.process_question(test_question)
print("Answer:", result["answer"])
print("Answer Rationale:", result["answer_rationale"])
print("Accuracy:", result["accuracy_metric"])
print("Accuracy Rationale:", result["rationale_metric"])

Searching table with query type: hybrid, table: context, query: Is BeatyPets open on public holidays?
Answer: BeatyPets is likely closed on public holidays.
Answer Rationale: The reasoning is based on the provided context, which states that BeatyPets is closed on Sundays and does not mention any special hours for public holidays.
Accuracy: 9
Accuracy Rationale: The answer is mostly accurate, but the use of "likely" introduces a slight uncertainty that is not explicitly supported by the context. A definitive statement would have


# Conclusions
The BeatyPets RAG System represents a sophisticated application of modern AI techniques, blending advanced natural language processing with robust data management to create an intelligent, responsive system. By leveraging DSPy for seamless integration and task management, along with LanceDB for efficient vector-based data retrieval, the system is able to deliver precise, context-aware answers to user queries.

This project not only demonstrates the power of combining state-of-the-art tools like DSPy and LanceDB but also provides a flexible framework that can be adapted to a wide range of domains beyond pet care. Whether used for customer support, virtual assistants, or knowledge management, the principles and architecture of this system offer a solid foundation for building intelligent, scalable, and user-friendly applications.

As AI continues to evolve, projects like this underscore the importance of integrating multiple technologies to achieve superior performance and usability. The BeatyPets RAG System is a testament to the potential of AI in enhancing user experiences through intelligent, contextually aware interactions.